# E2B Python SDK
The E2B Python SDK provides an interface for managing cloud environments for AI agents.

This SDK gives your agent a full cloud development environment that's sandboxed. That means:

- Access to Linux OS
- Using filesystem (create, list, and delete files and dirs)
- Run processes
- Sandboxed - you can run any code
- Access to the internet

These cloud environments are meant to be used for agents. Like a sandboxed playgrounds, where the agent can do whatever it wants.


## Installation

In [27]:
pip install e2b

2882.39s - pydevd: Sending message related to process being replaced timed-out after 5 seconds

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Usage

In [2]:
from e2b import Session

id = "Nodejs"

### Initialization

In [3]:
# You can use some of the predefined environments by using specific id:
# 'Nodejs', 'Bash', 'Python3', 'Java', 'Go', 'Rust', 'PHP', 'Perl', 'DotNET'
session = Session(id)

# Start a session and create a connection to it
await session.open()

# Close the session after you are done
await session.close()

### Filesystem

In [35]:
# Create, read and delete file

session = Session(id)
await session.open()

await session.filesystem.write("/tmp/test.txt", "Hello World!")

content = await session.filesystem.read("/tmp/test.txt")
print(content)

await session.filesystem.remove("/tmp/test.txt")

await session.close()

Hello World!


In [34]:
# List files in a directory, create a new directory and remove a directory

session = Session(id)
await session.open()

ls = await session.filesystem.list("/")
print([x.name for x in ls if x.name.startswith("test")])

await session.filesystem.make_dir("/test/new")

ls = await session.filesystem.list("/")
print([x.name for x in ls if x.name.startswith("test")])

await session.filesystem.remove("/test")

ls = await session.filesystem.list("/")
print([x.name for x in ls if x.name.startswith("test")])

await session.close()

[]
['test']
[]


In [10]:
# Watch directory for changes (WIP)

session = Session(id)
await session.open()

watcher = await session.filesystem.watch_dir("/")
watcher.add_event_listener(lambda event: print("Event", event))
await watcher.start()

await session.filesystem.write("/test.txt", "Hello World!")

await watcher.stop()

await session.close()

Event {'path': '//test.txt', 'name': 'test.txt', 'operation': 'Create', 'timestamp': 1691002491539042694, 'isDir': False}
Event {'path': '//test.txt', 'name': 'test.txt', 'operation': 'Write', 'timestamp': 1691002491539539655, 'isDir': False}


### Process

In [15]:
# Execute a command and get the output

session = Session(id)
await session.open()

proc = await session.process.start(
    "pwd",
    on_stdout=lambda data: print("Stdout", data),
    on_stderr=lambda data: print("Stderr", data),
    on_exit=lambda: print("Exit"),
    rootdir="/code",
)
print("Process session ID", proc.process_id)
await proc.finished

await session.close()

Process session ID lzE9nIb1RfcU
Stdout {'type': 'Stdout', 'line': '/code', 'timestamp': 1691002491468633866}
Exit


In [23]:
# Start command, send stdin to it and then kill it

session = Session(id)
await session.open()

proc = await session.process.start(
    "while IFS= read -r line; do echo \"$line\"; sleep 1; done",
    on_stdout=lambda data: print("Stdout", data),
    on_stderr=lambda data: print("Stderr", data),
    on_exit=lambda: print("Exit"),
    rootdir="/code",
)
await proc.send_stdin("marco\n")
await proc.kill()

await session.close()

Stdout {'type': 'Stdout', 'line': 'marco', 'timestamp': 1691002491598033549}
Exit


### Terminal

In [8]:
# Start and interact with a terminal session

session = Session(id)
await session.open()

term = await session.terminal.start(
    on_data=lambda data: print(data),
    on_exit=lambda: print("Exit"),
    cols=80,
    rows=24,
    rootdir="/code",
)
print("Terminal session ID", term.terminal_id)
await term.resize(80, 30)

await term.send_data("ls -a\n")

await term.kill()

await session.close()

Terminal session ID bDvviYtKKV89

/code $ 
ls -a

.  ..  package.json

/code $ 
Exit


In [13]:
# Execute one command inside of a terminal session

session = Session(id)
await session.open()

term = await session.terminal.start(
    on_data=lambda data: print("Data:", data),
    on_exit=lambda: print("Exit"),
    cols=80,
    rows=24,
    rootdir="/code",
    # If you specify a command, the terminal will be closed after the command finishes.
    cmd="echo Hello World",
)

await term.finished

await session.close()

Data: Hello World



### Ports and hostnames - connecting to environment

In [3]:
# Get notified when a port opens and how to get a public hostname for an open port in the cloud environment

import asyncio

session = Session(
    id,
    on_scan_ports=lambda ports: print("Open ports:", [port.port for port in ports]),
)
await session.open()

port = 8000
proc = await session.process.start(f"python3 -m http.server {port}")

hostname = session.get_hostname(port)
print(f"Hostname: https://{hostname}")

await asyncio.sleep(10)

await proc.kill()

await session.close()

Hostname: https://8000-siw5o0dw-fce131d5.ondevbook.com
Open ports: [49982, 49982, 22, 53]
Open ports: [49982, 49982, 53, 22, 8000]
Open ports: [49982, 8000, 22, 49982, 53]
Open ports: [49982, 8000, 53, 49982, 22]
Open ports: [49982, 53, 8000, 22, 49982]
Open ports: [49982, 8000, 22, 49982, 53]
Open ports: [49982, 8000, 49982, 22, 53]
Open ports: [49982, 8000, 22, 53, 49982]
Open ports: [49982, 8000, 49982, 53, 22, 8000, 8000]
Open ports: [49982, 53, 8000, 49982, 8000, 8000, 22]
Open ports: [49982, 22, 8000, 53, 8000, 8000, 49982]
